In [116]:

from typing import Dict, List
from graphviz import Digraph
import hashlib

class Pin:
    def __init__(self, name, value, component):
        self.name = name
        self.value = value
        self.component = component  # Reference to the component
        self.connections = []

    def connect(self, pin):
        if pin not in self.connections and pin is not self:
            self.connections.append(pin)

    def __eq__(self, other):
        return self.name == other.name

    def __repr__(self):
        return f"Pin({self.name})"

class Component:
    def __init__(self, name):
        self.name = name
        self.pins = {}

    def add_pin(self, pin_name, value=None):
        if pin_name not in self.pins:
            self.pins[pin_name] = Pin(pin_name, value, self)  # Set the component reference

    def get_pin(self, pin_name):
        return self.pins.get(pin_name)

    def __eq__(self, other):
        return self.name == other.name

    def __repr__(self):
        return f"Component({self.name})"

class Graph:
    def __init__(self):
        self.components = {}

    def add_component(self, component_name):
        if component_name not in self.components:
            self.components[component_name] = Component(component_name)

    def get_component(self, component_name):
        return self.components.get(component_name)

    def connect_pins(self, src_comp_name, src_pin_name, dst_comp_name, dst_pin_name):
        src_pin = self.get_component(src_comp_name).get_pin(src_pin_name)
        dst_pin = self.get_component(dst_comp_name).get_pin(dst_pin_name)
        if src_pin and dst_pin:
            src_pin.connect(dst_pin)

    def __repr__(self):
        return f"Graph({list(self.components.keys())})"
    
def get_color_for_component(component_name):
    # Create a hash object
    hash_object = hashlib.md5(component_name.encode())
    # Get the hex digest of the hash
    hash_hex = hash_object.hexdigest()
    # Convert the hex digest to an integer
    hash_int = int(hash_hex, 16)
    # Generate RGB values from the hash integer
    r = (hash_int >> 16) & 0xFF
    g = (hash_int >> 8) & 0xFF
    b = hash_int & 0xFF
    # Return the color as a hex string
    return f'#{r:02X}{g:02X}{b:02X}'



def generate_dot_file(graph):
    dot = Digraph()
    dot.attr(fontname="Helvetica,Arial,sans-serif")
    dot.node_attr.update(fontname="Helvetica,Arial,sans-serif", fontsize="16", shape="ellipse")
    dot.edge_attr.update(fontname="Helvetica,Arial,sans-serif")
    dot.graph_attr.update(rankdir="LR", overlap="false")

    for comp in graph.components.values():
        color = get_color_for_component(comp.name)
        label = f'<<table border="0" cellborder="1" cellspacing="0">\n\t\t\t<tr><td colspan="1" bgcolor="{color}"><b>{comp.name}</b></td></tr>\n'
        
        for pin in comp.pins.values():
            label += f'\t\t\t<tr><td port="{pin.name}">{pin.name}{" = " + str(pin.value) if pin.value is not None else ""}</td></tr>' + '\n'
        label += "\t\t\t</table>>"
        dot.node(str(comp.name), label=label, shape="none")

    for comp_name, comp in graph.components.items():
        for pin_name, pin in comp.pins.items():
            for connected_pin in pin.connections:
                dst_pin_ref = f'{comp_name}:{pin_name}'
                src_pin_ref = f'{connected_pin.component.name}:{connected_pin.name}'
                dot.edge(src_pin_ref, dst_pin_ref)

    return dot

In [117]:
# Define the components and signals as given in your test.ipynb
components = {
    "ls0": ["mot_temp", "dc_volt", "hv_temp", "fault_in", "d_cmd", "q_cmd", "pos", "phase_mode", "en", "dac"],
    "io0": ["mot_temp", "udc", "hv_temp", "fault", "led", "hv_en", "dac", "iu", "iv", "iw", "ignore_fault_pin"],
    "dq0": ["pos", "mode", "u", "v", "w", "y", "d", "q"],
    "curpid0": ["id_cmd", "iq_cmd", "id_fb", "iq_fb", "ud", "uq", "r", "ld", "lq", "psi", "cur_bw", "ff", "kind", "max_cur", "pwm_volt", "vel", "en", "cmd_mode"],
    "idq0": ["pos", "mode", "u", "v", "w", "d", "q"],
    "svm0": ["u", "v", "w", "su", "sv", "sw", "udc"],
    "hv0": ["udc", "iu", "iv", "iw", "arr"]
}

signals = [
    ("ls0.mot_temp", "io0.mot_temp"),
    ("ls0.dc_volt", "io0.udc"),
    ("ls0.hv_temp", "io0.hv_temp"),
    ("ls0.fault_in", "io0.fault"),
    ("io0.led", "ls0.fault"),
    ("curpid0.id_cmd", "ls0.d_cmd"),
    ("curpid0.iq_cmd", "ls0.q_cmd"),
    ("idq0.pos", "ls0.pos"),
    ("idq0.mode", "ls0.phase_mode"),
    ("dq0.pos", "ls0.pos"),
    ("dq0.mode", "ls0.phase_mode"),
    ("io0.hv_en", "ls0.en"),
    ("io0.dac", "ls0.dac"),
    ("hv0.udc", "io0.udc"),
    ("dq0.u", "io0.iu"),
    ("dq0.v", "io0.iv"),
    ("dq0.w", "io0.iw"),
    ("svm0.u", "idq0.u"),
    ("svm0.v", "idq0.v"),
    ("svm0.w", "idq0.w"),
    ("hv0.u", "svm0.su"),
    ("hv0.v", "svm0.sv"),
    ("hv0.w", "svm0.sw"),
    ("svm0.udc", "io0.udc"),
    ("curpid0.id_fb", "dq0.d"),
    ("curpid0.iq_fb", "dq0.q"),
    ("ls0.id_fb", "dq0.d"),
    ("ls0.iq_fb", "dq0.q"),
    ("ls0.ud_fb", "curpid0.ud"),
    ("ls0.uq_fb", "curpid0.uq"),
    ("ls0.y", "dq0.y"),
    ("ls0.u_fb", "io0.u"),
    ("ls0.v_fb", "io0.v"),
    ("ls0.w_fb", "io0.w"),
    ("idq0.d", "curpid0.ud"),
    ("idq0.q", "curpid0.uq"),
    ("curpid0.r", "ls0.r"),
    ("curpid0.ld", "ls0.l"),
    ("curpid0.lq", "ls0.l"),
    ("curpid0.psi", "ls0.psi"),
    ("curpid0.cur_bw", "ls0.cur_bw"),
    ("curpid0.ff", "ls0.cur_ff"),
    ("curpid0.kind", "ls0.cur_ind"),
    ("curpid0.max_cur", "ls0.max_cur"),
    ("curpid0.pwm_volt", "ls0.pwm_volt"),
    ("curpid0.vel", "ls0.vel"),
    ("curpid0.en", "ls0.en"),
    ("curpid0.cmd_mode", "ls0.cmd_mode"),
    ("hv0.arr", "ls0.arr"),
    ("io0.ignore_fault_pin", "ls0.ignore_fault_pin")
]

# Initialize the graph
graph = Graph()

# Add components to the graph
for comp_name, pins in components.items():
    graph.add_component(comp_name)
    for pin in pins:
        graph.get_component(comp_name).add_pin(pin)

# Connect pins according to the signals list
for src_signal, dst_signal in signals:
    src_comp_name, src_pin_name = src_signal.split('.')
    dst_comp_name, dst_pin_name = dst_signal.split('.')

    graph.connect_pins(src_comp_name, src_pin_name, dst_comp_name, dst_pin_name)

# Generate the DOT file
dot = generate_dot_file(graph)

# Save to a file and render as an image
dot.render('graph', format='dot')
dot.render('graph', format='png')

'graph.png'